# Gleaning: Iterative Self-Critique for LLMs

This notebook demonstrates the *gleaning* pattern—an iterative draft, critique, and revise loop for improving LLM outputs at inference time. Inspired by Anthropic's Constitutional AI and recent research, this approach can boost quality and safety without extra model training.

## What is Gleaning?

Gleaning is an inference-time self-review loop for LLMs:
1. **Draft**: Generate an initial output.
2. **Critique**: The model reviews its own output and suggests improvements.
3. **Revise**: The model revises its output based on the critique.
4. **Repeat**: Loop until a quality gate is passed or a maximum number of iterations is reached.

This mimics human editing and can be implemented with any LLM API.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<API_KEY>"

In [2]:
from mirascope import llm, prompt_template
from pydantic import BaseModel

class Critique(BaseModel):
 feedback: str
 needs_improvement: bool

@llm.call(provider="openai", model="gpt-4o-mini")
@prompt_template("Write a polite email declining a job offer for: {role}")
def draft(role: str) -> str: ...

@llm.call(provider="openai", model="gpt-4o-mini", response_model=Critique)
@prompt_template("""
                    Evaluate the email and suggest improvements.
                    Role: {role}
                    Email: {email}
                """)
def critique(role: str, email: str) -> Critique: ...

@llm.call(provider="openai", model="gpt-4o-mini")
@prompt_template("""
                    Revise the email using this feedback.
                    Role: {role}
                    Email: {email}
                    Feedback: {feedback}
                """)
def revise(role: str, email: str, feedback: str) -> str: ...

## The Glean Loop

This function runs the draft-critique-revise loop up to a maximum number of iterations, or until the critique says no further improvement is needed.

In [3]:
def glean(role: str, max_iter: int = 3) -> str:
    email = draft(role)
    for _ in range(max_iter):
        fb = critique(role, email)
        print("Feedback: ", fb.feedback, end="\n\n--------------------------------\n\n")
        if not fb.needs_improvement:
            print("No improvement needed")
            return email
        email = revise(role, email, fb.feedback)
    return email

## Example: Polite Email Declining a Job Offer

Let's see the gleaning loop in action for the role of 'Data Scientist'.

In [4]:
final_result = glean('Data Scientist')
print(final_result.response.choices[0].message.content)

Feedback:  The email is polite and expresses gratitude, which is great. However, it could be improved by being more concise and specific. For example, mentioning what specifically led to the decision to decline (without going into too much detail) might provide clarity. Additionally, a positive remark about the team or a specific project discussed during the interviews could strengthen the connection. Finally, ending with an invitation to keep in touch or a wish that their search for the right candidate is successful could leave a better impression.

--------------------------------

Feedback:  The email is polite and expresses gratitude; however, it could be improved by mentioning specific reasons for declining the offer to provide closure. Including a stronger expression of interest in potential future opportunities could also enhance the connection. Furthermore, consider personalizing the email with specific details about the interactions or insights from the interview that were par

## Discussion

- In a real-world setting, the `draft`, `critique`, and `revise` functions would call an LLM API (e.g., OpenAI, Anthropic) with appropriate prompts.
- The critique step can use a structured schema (e.g., Pydantic) to ensure reliable feedback.
- This pattern is used in Anthropic's Constitutional AI, which showed large improvements in safety and quality ([Bai et al., 2022](https://arxiv.org/abs/2212.08073)).
- You can adapt this notebook to your own tasks by changing the prompts and feedback logic.

## References
- Bai, Yuntao, et al. "Constitutional AI: Harmlessness from AI Feedback." arXiv preprint arXiv:2212.08073 (2022). [arxiv.org/abs/2212.08073](https://arxiv.org/abs/2212.08073)
- Madaan, Aman, et al. "Self-Refine: Iterative Refinement with Self-Feedback." arXiv preprint arXiv:2303.17651 (2023). [arxiv.org/abs/2303.17651](https://arxiv.org/abs/2303.17651)
